# 이분 그래프 기반 추천 시스템 테스트

In [90]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

import pandas as pd
interactions = pd.DataFrame(columns=['user_id', 'movie_id'])
interactions['user_id'] = [1, 1, 1]
interactions['movie_id'] = [3, 4, 5951]
users = pd.DataFrame(columns=['user_id'])
users['user_id'] = [1]

# 1. 데이터 전처리
movies = np.load("final_features_without_directors_0511.npy")

num_movies = 9525
num_users = users.shape[0]
num_features = 406

# 2. 이분 그래프 생성
edges = interactions[['user_id', 'movie_id']].values.transpose()
edge_index = torch.tensor(edges, dtype=torch.long)
x = torch.from_numpy(movies).float()

# 레이블 값 생성: 유저-영화 간 에지가 있으면 1, 없으면 0
num_edges = edge_index.size(1)
edge_labels = torch.zeros(num_movies, dtype=torch.float)
for i in range(num_edges):
    edge_labels[edge_index[1, i]] = 1
    
graph_data = Data(x=x, edge_index=edge_index, y=edge_labels.unsqueeze(1))

# 3. Graph Attention Network (GAT) 모델 구현
class GATRecommender(nn.Module):
    def __init__(self, num_features, hidden_channels, num_heads):
        super().__init__()
        self.conv1 = GATConv(num_features, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, num_heads, concat=False, dropout=0.6)
        self.lin = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = self.lin(x)
        return x

# 4. 모델 학습
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATRecommender(num_features, hidden_channels=128, num_heads=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(graph_data.x.to(device), graph_data.edge_index.to(device))
    loss = criterion(out, graph_data.y.to(device))
    loss.backward()
    optimizer.step()

# 5. 추천 시스템 구현
def recommend_movies(user_id, top_k=5):
    user_movies = interactions[interactions['user_id'] == user_id]['movie_id'].values
    # 사용자가 아직 시청하지 않은 영화들
    non_user_movies = np.array([movie for movie in range(num_movies) if movie not in user_movies])
    
    # 사용자와 사용자가 보지 않은 모든 영화 사이의 잠재적 간선 생성
    user_edges = torch.tensor([[user_id] * len(non_user_movies), non_user_movies], dtype=torch.long).to(device)
    
    with torch.no_grad():
        model.eval()
        # 사용자와 사용자가 보지 않은 모든 영화 사이의 잠재적 간선에 대해 예측 수행
        out = model(graph_data.x.to(device), user_edges)
    
    # 상위 k개의 추천 영화 인덱스 추출
    recommended_movies = non_user_movies[torch.topk(out.squeeze(), top_k).indices.cpu().numpy()]
    return recommended_movies

In [91]:
recommend_movies(1, top_k=5)

array([8562, 5206, 9209, 5723, 8954])

# Check

In [47]:
import pandas as pd

movie_metadata = pd.read_excel("cinemate_data_0511.xlsx")
movie_metadata

,movie_id,rating,backdrop_path,original_title,title,release_date,poster_path,overview
0,1146410,0.000,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,여행자의 필요,여행자의 필요,2024-04-24,/fNhyJp2fy47vg66LZ8aA2b0tsVQ.jpg,"어디서 온지 모르는 이 사람은 불란서에서 왔다고 하고, 어린애 피리를 근린공원에서 ..."
1,876127,6.800,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,도그데이즈,도그데이즈,2024-02-07,/bl3YQHEENj4B80Ku3PkTJWWLnj5.jpg,깔끔한 성격의 계획형 싱글남 민상. 영끌까지 모아 산 건물을 개똥밭으로 만드는 세입...
2,1072342,5.729,/klb3yODwKMRle19EcvdeSatEZL7.jpg,Night Swim,나이트 스윔,2024-01-03,/4Q20D0HCYNHwvTna4OXJNCDvyI3.jpg,넓은 수영장이 있는 새집으로 이사 온 후 더할 나위 없는 행복을 느끼고 있는 ‘레이...
3,838209,7.500,/aINel9503ompOlGKn4sIVMg09Un.jpg,파묘,파묘,2024-02-22,/tw0i3kkmOTjDjGFZTLHKhoeXVvA.jpg,"미국 LA, 거액의 의뢰를 받은 무당 화림과 봉길은 기이한 병이 대물림되는 집안의 ..."
4,1017163,8.600,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,범죄도시 4,범죄도시 4,2024-04-24,/h1YarEjeYurkAwXgfY1RDMVCiin.jpg,"신종 마약 사건 3년 뒤, 괴물형사 마석도와 서울 광수대는 배달앱을 이용한 마약 판..."
...,...,...,...,...,...,...,...,...
9520,67319,6.500,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,So This Is Paris,여기는 파리,1926-07-31,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,폴과 수자나 부부는 조용한 마을에서 행복하게 살고 있다. 그러던 중 화려한 댄서들이...
9521,5991,7.785,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,Der letzte Mann,마지막 웃음,1924-12-23,/woR2d4zkWgCPlvwQ9kMHcrdiGPJ.jpg,은발의 멋진 구렛나루 수염을 쓰다듬으며 금빛 단추의 제복을 뽐내는 한 호텔 도어맨이...
9522,28974,6.731,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,A Woman of Paris: A Drama of Fate,파리의 여인,1923-10-01,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,마리(Marie St. Clair: 에드나 퍼비안스 분)는 계부의 반대를 피해 애인...
9523,175472,5.800,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,Vier um die Frau,여자를 둘러싼 넷,1921-02-03,/83nTPJQnzrn78ppVf5L5CEOIkIg.jpg,해리 유켐은 그의 사랑하는 아내에게 가짜와 장물을 거래하는 지하 세계의 중간거래상들...


In [92]:
movie_metadata.iloc[5723].overview

'세기의 명탐정_“범죄는 흔하다. 그러나 논리는 흔치 않다” 셜록 홈즈, 천재적인 추리 능력과 주먹의 힘까지 갖추고 친구 왓슨 박사 와 함께 치밀하게 얽힌 미스터리 속에서 진실을 찾아내는 명탐정. 그에게 이제껏 경험하지 못했던 최대의 위협이자 지금껏 그토록 갈구했던 진정한 모험이 몰려오고 있었다. 예고된 살인_“홈즈, 당신과 나는 자연 법칙을 뒤집어 놓을 여정에 올랐소”다섯 명의 여인들이 종교 의식의 제물로 끔찍하게 살해되는 사건이 잇따라 발생하고, 홈즈와 왓슨은 간발의 차이로 마지막 희생자가 될뻔한 여인을 구한다. 범인은 비밀 종교집단 소속의 블랙우드. 붙잡힌 블랙우드는 사형 집행일이 다가올수록 강력한 어둠의 힘을 발휘하고, 자신의 죽음은 계획의 일부에 지나지 않는다며 홈즈에게 경고한다. 죽은 자의 부활_“증거 없는 이론은 위험하네. 사실을 이론에 맞추려 들테니” 블랙우드의 경고는 현실로 나타나고, 죽었던 그의 부활은 도시 전체를 공포 속에 몰아넣는다. 그러나 이것은 홈즈에게는 게임의 시작일 뿐. 블랙우드의 치명적인 음모를 저지하기 위해 홈즈와 왓슨은 고대의 신비한 주술과 현대의 경이로운 신기술이 혼재한 세계로 뛰어든다. 진정한 목적_“가장 사소한 단서가 가장 중요한 증거가 되는 법이지”그러나 홈즈가 해결해야 할 것은 사건만이 아니었다. 그의 앞에 헤어진 연인 아이린 이 등장해 적인지 아닌지 모호한 행동으로 그를 더욱 혼란에 빠뜨린다. 한편, 최강의 콤비 플레이로 사건을 파헤치던 홈즈와 왓슨은 단서들이 공통의 연결고리로 어떤 징후를 나타낸다는 것을 발견하고, 이것이 세상을 파멸시킬 거대한 음모였음을 알게 되는데… 정확한 논리를 무기로, 일격의 주먹을 방어막으로, 세상을 구할 홈즈의 추리가 시작된다!'

In [95]:
movie_metadata.iloc[9209]

movie_id                                                      42787
rating                                                          6.2
backdrop_path                      /aTmbTmoXqc85rFtKc6n12XwWUaT.jpg
original_title                                       Lady in a Cage
title                                                       새장속의 여인
release_date                                             1964-06-10
poster_path                        /f0eafb0uhYhBTvA2MRB1vN0yJL7.jpg
overview          자신의 집에서 실내 엘리베이터에 갇혀 버린 후, 집안에 불법 침입한 괴한들로부터 위...
Name: 9209, dtype: object

In [53]:
import pandas as pd

final_metadata = pd.read_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")
final_metadata

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,ordering,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,tt31015344,A Traveler's Needs,Yeohaengjaui Pilyo,2024,90,Drama,6.5,75,nm0393254,1,...,0.016458,-0.023006,-0.050298,0.037739,-0.076090,-0.010432,-0.058701,0.017649,0.035267,-0.023228
1,tt30911335,Dog Days,Dogeudeijeu,2024,120,Drama,7.8,31,nm15730187,1,...,-0.034701,-0.008065,-0.024117,-0.003312,0.038821,-0.030186,0.019926,0.074339,-0.018989,0.055851
2,tt9682428,Night Swim,Night Swim,2024,98,"Horror,Thriller",4.7,12654,nm3241399,1,...,0.003738,-0.049932,-0.121888,-0.054388,-0.020564,0.050116,0.021484,0.052849,0.035944,0.025606
3,tt27802490,Exhuma,Pamyo,2024,134,"Horror,Mystery,Thriller",7.4,495,nm6940206,1,...,0.051392,-0.002096,0.044035,0.059909,-0.044593,-0.032423,0.081002,0.020807,0.021268,0.008938
4,tt27811040,The Roundup: Punishment,Beomjoedosi4,2024,109,"Action,Crime,Thriller",8.8,101,nm2441168,10,...,0.041042,-0.039577,0.034051,0.041895,-0.132000,-0.051895,0.089292,-0.132800,-0.011520,-0.006363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,tt0017409,So This Is Paris,So This Is Paris,1926,60,Comedy,7.1,746,nm0523932,1,...,-0.052198,-0.056215,-0.065513,0.068392,0.040504,-0.013506,-0.068042,0.001189,-0.021429,-0.103460
9521,tt0015064,The Last Laugh,Der letzte Mann,1924,88,Drama,8.0,15105,nm0003638,1,...,0.029271,-0.052846,0.032112,0.051352,-0.038684,-0.013877,0.074433,-0.020061,-0.008468,0.026134
9522,tt0014624,A Woman of Paris: A Drama of Fate,A Woman of Paris: A Drama of Fate,1923,82,"Drama,Romance",6.9,6104,nm0000122,10,...,0.018064,-0.050176,-0.045593,0.036943,-0.046362,-0.033189,-0.061279,-0.036766,0.073398,-0.105024
9523,tt0012806,Four Around the Woman,Vier um die Frau,1921,52,Drama,5.9,581,nm0000485,1,...,-0.047176,-0.100745,0.056739,0.063528,-0.031887,-0.028223,-0.002628,-0.041470,0.026425,0.023838


In [87]:
final_metadata.iloc[5206]

tconst            tt2025667
primaryTitle         Smiley
originalTitle        Smiley
startYear              2012
runtimeMinutes           95
                    ...    
embedding_379      0.117943
embedding_380      0.145757
embedding_381      0.061894
embedding_382      0.040974
embedding_383       0.00945
Name: 5206, Length: 396, dtype: object

In [ ]:
# [3, 4, 5951]를 좋아하는 사용자에게 추천하는 영화
# array([9209, 2630, 8562, 8954, 5206]) 첫번째 추천 결과
# array([8562, 5206, 9209, 5723, 8954]) 두번째 추천 결과